In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import csv
from bs4 import BeautifulSoup

browser = Chrome()
time.sleep(2)

movie = []

for j in range(2013,2023):
    # 년마다 데이터 수집 시작
    url = f"https://movie.daum.net/ranking/boxoffice/yearly?date={j}"
    browser.get(url)
    time.sleep(2)
    
    for i in range(1,51):
        # 잘못된 거 수집하는 지 확인
        누적완료 = {}
        
        # 연도와 순위 저장
        
        누적완료["연도"] = j

        누적완료["순위"] = i
        
        title = browser.find_element(By.XPATH,f"/html/body/div[2]/main/article/div/div[2]/ol/li[{i}]/div/div[2]/strong/a")
        누적완료["제목"] = title.text
        time.sleep(2)
        
        # N년도 목록에 있는 영화에 들어가기
        
        btn_elem = browser.find_element(By.XPATH,f"/html/body/div[2]/main/article/div/div[2]/ol/li[{i}]/div/div[2]/strong/a")
        btn_elem.click()
        time.sleep(2)
        
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")
        cls = soup.select(".inner_cont:nth-of-type(1) > .list_cont")
        
        # 감독 
        
        try:
            director = browser.find_element(By.XPATH,"/html/body/div[2]/main/article/div/div[2]/div[2]/div[2]/ul/li[1]/div/div/strong/a")
        except NoSuchElementException:
            누적완료["감독"] = "확인불가"
        else:
            누적완료["감독"] = director.text
        
        # 주연1
        try:
            actor1 = browser.find_element(By.XPATH,"/html/body/div[2]/main/article/div/div[2]/div[2]/div[2]/ul/li[2]/div/div/strong/a")
        except NoSuchElementException:
            누적완료["주연1"] = "확인불가"
        else:
            누적완료["주연1"] = actor1.text
        
        # 주연2    
        try:
            actor2 = browser.find_element(By.XPATH,"/html/body/div[2]/main/article/div/div[2]/div[2]/div[2]/ul/li[3]/div/div/strong/a")
        except NoSuchElementException:
            누적완료["주연2"] = "확인불가"
        else:
            누적완료["주연2"] = actor2.text
        
        # 개봉일
        try:
            when = browser.find_element(By.XPATH,"/html/body/div[2]/main/article/div/div[1]/div[2]/div[2]/div[1]/dl[1]/dd")
        except NoSuchElementException:
            누적완료["개봉일"] = "확인불가"
        else:
            누적완료["개봉일"] = when.text
        
        # 관객수
        try:
            audience = browser.find_element(By.XPATH,f"/html/body/div[2]/main/article/div/div[1]/div[2]/div[2]/div[2]/dl[2]/dd")
        except NoSuchElementException:
            누적완료["관객수"] = 0
        else:
            # 정수형으로 만들기 위해 '명'과 ',' 제거
            aud = audience.text.replace("명","")
            audie = aud.replace(",","")
            누적완료["관객수"] = int(audie)
        
        # 장르
        try:
            # 몇몇 영화에는 재개봉 항목이 있어서 잘못된 것을 수집 방지
            if len(cls) == 6:
                genre = browser.find_element(By.XPATH,f"/html/body/div[2]/main/article/div/div[1]/div[2]/div[2]/div[1]/dl[3]/dd")
            else:
                genre = browser.find_element(By.XPATH,f"/html/body/div[2]/main/article/div/div[1]/div[2]/div[2]/div[1]/dl[2]/dd")
        except NoSuchElementException:
            누적완료["장르"] = "확인불가"
        else:
            누적완료["장르"] = genre.text
        
        # 수상
        try:
            award = browser.find_element(By.XPATH,"/html/body/div[2]/main/article/div/div[1]/div[2]/div[2]/div[2]/dl[3]/dd/a")
        except NoSuchElementException:
            누적완료["수상"] = "확인불가"
        else:
            # 회와 년도 제거
            aw = award.text
            aw = list(aw)
            aw[0:3] = ''
            aw[-6:] = ''
            누적완료["수상"] = f"{''.join(aw).strip()}" 
        
        # 확인용
        print(f"누적완료:{누적완료}")
        # 정보를 딕셔너리화 시킨고 리스트에 담는다.
        movie.append(누적완료)
        
        # 영화 정보 수집 완료되면 뒤로 가기
        browser.back()

# 영화를 수집후 csv에 저장        
        
f = open('영화 박스오피스(2013-2022).csv','w',newline='')
wr = csv.writer(f)
wr.writerow(['연도','순위','제목','감독','주연1','주연2','개봉일','관객수','장르','수상'])
for i in range(len(movie)):
    wr.writerow([movie[i]['연도'],movie[i]['순위'],movie[i]['제목'],movie[i]['감독'],movie[i]['주연1'],movie[i]['주연2'],movie[i]['개봉일'],movie[i]['관객수'],movie[i]['장르'],movie[i]['수상']])
f.close()

누적완료:{'연도': 2013, '순위': 1, '제목': '7번방의 선물', '감독': '이환경', '주연1': '류승룡', '주연2': '박신혜', '개봉일': '2013.01.23', '관객수': 12812186, '장르': '드라마', '수상': '황금촬영상'}
누적완료:{'연도': 2013, '순위': 2, '제목': '설국열차', '감독': '봉준호', '주연1': '크리스 에반스', '주연2': '송강호', '개봉일': '2013.08.01', '관객수': 9354547, '장르': 'SF/액션/드라마', '수상': '미국비평가협회상'}
누적완료:{'연도': 2013, '순위': 3, '제목': '관상', '감독': '한재림', '주연1': '송강호', '주연2': '이정재', '개봉일': '2013.09.11', '관객수': 9135806, '장르': '시대극', '수상': '백상예술대상'}
누적완료:{'연도': 2013, '순위': 4, '제목': '아이언맨 3', '감독': '셰인 블랙', '주연1': '로버트 다우니 주니어', '주연2': '기네스 팰트로', '개봉일': '2013.04.25', '관객수': 9001679, '장르': '액션/SF', '수상': '확인불가'}
누적완료:{'연도': 2013, '순위': 5, '제목': '베를린', '감독': '류승완', '주연1': '하정우', '주연2': '한석규', '개봉일': '2013.01.30', '관객수': 7166688, '장르': '액션/드라마', '수상': '부산영화평론가협회상'}
누적완료:{'연도': 2013, '순위': 6, '제목': '은밀하게 위대하게', '감독': '장철수', '주연1': '김수현', '주연2': '박기웅', '개봉일': '2013.06.05', '관객수': 6959083, '장르': '액션/드라마/코미디', '수상': '백상예술대상'}
누적완료:{'연도': 2013, '순위': 7, '제목': '변호인', '감독': '양우석', '주연1': '송강호'

누적완료:{'연도': 2014, '순위': 3, '제목': '인터스텔라', '감독': '크리스토퍼 놀란', '주연1': '매튜 맥커너히', '주연2': '앤 해서웨이', '개봉일': '2014.11.06', '관객수': 10342523, '장르': 'SF', '수상': '맥스무비 최고의 영화상'}
누적완료:{'연도': 2014, '순위': 4, '제목': '해적: 바다로 간 산적', '감독': '이석훈', '주연1': '김남길', '주연2': '손예진', '개봉일': '2014.08.06', '관객수': 8666208, '장르': '어드벤처', '수상': '백상예술대상'}
누적완료:{'연도': 2014, '순위': 5, '제목': '수상한 그녀', '감독': '황동혁', '주연1': '심은경', '주연2': '나문희', '개봉일': '2014.01.22', '관객수': 8662080, '장르': '코미디/드라마', '수상': '맥스무비 최고의 영화상'}
누적완료:{'연도': 2014, '순위': 6, '제목': '변호인', '감독': '양우석', '주연1': '송강호', '주연2': '김영애', '개봉일': '2013.12.18', '관객수': 11375399, '장르': '드라마', '수상': '청룡영화상'}
누적완료:{'연도': 2014, '순위': 7, '제목': '국제시장', '감독': '윤제균', '주연1': '황정민', '주연2': '김윤진', '개봉일': '2014.12.17', '관객수': 14265222, '장르': '드라마', '수상': '한국영화제작가협회상'}
누적완료:{'연도': 2014, '순위': 8, '제목': '트랜스포머: 사라진 시대', '감독': '마이클 베이', '주연1': '마크 월버그', '주연2': '니콜라 펠츠', '개봉일': '2014.06.25', '관객수': 5295836, '장르': 'SF', '수상': '골든라즈베리시상식'}
누적완료:{'연도': 2014, '순위': 9, '제목': '군도: 민란의 시대', '

누적완료:{'연도': 2015, '순위': 5, '제목': '내부자들', '감독': '우민호', '주연1': '이병헌', '주연2': '조승우', '개봉일': '2015.11.19', '관객수': 7073581, '장르': '범죄/드라마', '수상': '대종상영화제'}
누적완료:{'연도': 2015, '순위': 6, '제목': '사도', '감독': '이준익', '주연1': '송강호', '주연2': '유아인', '개봉일': '2015.09.16', '관객수': 6247745, '장르': '시대극', '수상': '인도국제영화제'}
누적완료:{'연도': 2015, '순위': 7, '제목': '킹스맨 : 시크릿 에이전트', '감독': '매튜 본', '주연1': '콜린 퍼스', '주연2': '태런 에저튼', '개봉일': '2015.02.11', '관객수': 6129681, '장르': '스릴러', '수상': '확인불가'}
누적완료:{'연도': 2015, '순위': 8, '제목': '미션 임파서블: 로그네이션', '감독': '크리스토퍼 맥쿼리', '주연1': '톰 크루즈', '주연2': '제레미 레너', '개봉일': '2015.07.30', '관객수': 6126488, '장르': '액션/어드벤처', '수상': '확인불가'}
누적완료:{'연도': 2015, '순위': 9, '제목': '연평해전', '감독': '김학순', '주연1': '김무열', '주연2': '진구', '개봉일': '2015.06.24', '관객수': 6045726, '장르': '드라마', '수상': '휴스턴국제영화제'}
누적완료:{'연도': 2015, '순위': 10, '제목': '쥬라기 월드', '감독': '콜린 트레보로우', '주연1': '크리스 프랫', '주연2': '브라이스 달라스 하워드', '개봉일': '2015.06.11', '관객수': 5547463, '장르': '액션/어드벤처/SF', '수상': 'MTV 영화 & TV 상'}
누적완료:{'연도': 2015, '순위': 11, '제목': '검은 

누적완료:{'연도': 2016, '순위': 7, '제목': '럭키', '감독': '이계벽', '주연1': '유해진', '주연2': '이준', '개봉일': '2016.10.13', '관객수': 6975631, '장르': '코미디', '수상': '대한민국 대학영화제'}
누적완료:{'연도': 2016, '순위': 8, '제목': '곡성', '감독': '나홍진', '주연1': '곽도원', '주연2': '황정민', '개봉일': '2016.05.12', '관객수': 6879989, '장르': '미스터리/스릴러/드라마', '수상': '황금촬영상'}
누적완료:{'연도': 2016, '순위': 9, '제목': '덕혜옹주', '감독': '허진호', '주연1': '손예진', '주연2': '박해일', '개봉일': '2016.08.03', '관객수': 5599995, '장르': '시대극/드라마', '수상': '황금촬영상'}
누적완료:{'연도': 2016, '순위': 10, '제목': '닥터 스트레인지', '감독': '스콧 데릭슨', '주연1': '베네딕트 컴버배치', '주연2': '레이첼 맥아담스', '개봉일': '2016.10.26', '관객수': 5447269, '장르': '어드벤처/판타지/액션/SF', '수상': '할리우드 필름어워즈'}
누적완료:{'연도': 2016, '순위': 11, '제목': '마스터', '감독': '조의석', '주연1': '이병헌', '주연2': '강동원', '개봉일': '2016.12.21', '관객수': 7150586, '장르': '범죄/액션', '수상': '확인불가'}
누적완료:{'연도': 2016, '순위': 12, '제목': '주토피아', '감독': '바이런 하워드', '주연1': '리치 무어', '주연2': '지니퍼 굿윈', '개봉일': '2016.02.17', '관객수': 4707962, '장르': '애니메이션/액션/어드벤처', '수상': '아카데미시상식'}
누적완료:{'연도': 2016, '순위': 13, '제목': '신비한 동물사전', '

누적완료:{'연도': 2017, '순위': 9, '제목': '미녀와 야수', '감독': '빌 콘돈', '주연1': '엠마 왓슨', '주연2': '댄 스티븐스', '개봉일': '2017.03.16', '관객수': 5156372, '장르': '판타지/뮤지컬/로맨스/멜로', '수상': '할리우드 필름어워즈'}
누적완료:{'연도': 2017, '순위': 10, '제목': '킹스맨: 골든 서클', '감독': '매튜 본', '주연1': '콜린 퍼스', '주연2': '줄리안 무어', '개봉일': '2017.09.27', '관객수': 4945484, '장르': '액션/어드벤처/코미디', '수상': '확인불가'}
누적완료:{'연도': 2017, '순위': 11, '제목': '토르: 라그나로크', '감독': '타이카 와이티티', '주연1': '톰 히들스턴', '주연2': '케이트 블란쳇', '개봉일': '2017.10.25', '관객수': 4858572, '장르': '액션/판타지', '수상': '확인불가'}
누적완료:{'연도': 2017, '순위': 12, '제목': '꾼', '감독': '장창원', '주연1': '현빈', '주연2': '유지태', '개봉일': '2017.11.22', '관객수': 4018341, '장르': '범죄', '수상': '확인불가'}
누적완료:{'연도': 2017, '순위': 13, '제목': '강철비', '감독': '양우석', '주연1': '정우성', '주연2': '곽도원', '개봉일': '2017.12.14', '관객수': 4452900, '장르': '액션/드라마', '수상': '한국영화평론가협회상'}
누적완료:{'연도': 2017, '순위': 14, '제목': '남한산성', '감독': '황동혁', '주연1': '이병헌', '주연2': '김윤석', '개봉일': '2017.10.03', '관객수': 3849159, '장르': '드라마', '수상': '디렉터스 컷 어워즈'}
누적완료:{'연도': 2017, '순위': 15, '제목': '미이라', '감독'

누적완료:{'연도': 2018, '순위': 10, '제목': '완벽한 타인', '감독': '이재규', '주연1': '유해진', '주연2': '조진웅', '개봉일': '2018.10.31', '관객수': 5294155, '장르': '코미디/드라마', '수상': '한국영화평론가협회상'}
누적완료:{'연도': 2018, '순위': 11, '제목': '1987', '감독': '장준환', '주연1': '김윤석', '주연2': '하정우', '개봉일': '2017.12.27', '관객수': 7232527, '장르': '드라마', '수상': '한국영화제작가협회상'}
누적완료:{'연도': 2018, '순위': 12, '제목': '독전', '감독': '이해영', '주연1': '조진웅', '주연2': '류준열', '개봉일': '2018.05.22', '관객수': 5064323, '장르': '범죄/액션', '수상': '제천국제음악영화제'}
누적완료:{'연도': 2018, '순위': 13, '제목': '공작', '감독': '윤종빈', '주연1': '황정민', '주연2': '이성민', '개봉일': '2018.08.08', '관객수': 4975517, '장르': '드라마', '수상': '황금촬영상'}
누적완료:{'연도': 2018, '순위': 14, '제목': '베놈', '감독': '루벤 플레셔', '주연1': '톰 하디', '주연2': '미셀 윌리엄스', '개봉일': '2018.10.03', '관객수': 3888096, '장르': '액션/SF/스릴러', '수상': '확인불가'}
누적완료:{'연도': 2018, '순위': 15, '제목': '암수살인', '감독': '김태균', '주연1': '김윤석', '주연2': '주지훈', '개봉일': '2018.10.03', '관객수': 3789553, '장르': '범죄', '수상': '한중국제영화제'}
누적완료:{'연도': 2018, '순위': 16, '제목': '데드풀 2', '감독': '데이비드 레이치', '주연1': '라이언 레이놀즈', '주

누적완료:{'연도': 2019, '순위': 12, '제목': '라이온 킹', '감독': '존 파브로', '주연1': '도널드 글로버', '주연2': '비욘세', '개봉일': '2019.07.17', '관객수': 4743295, '장르': '어드벤처/드라마/가족', '수상': '새틀라이트시상식'}
누적완료:{'연도': 2019, '순위': 13, '제목': '나쁜 녀석들: 더 무비', '감독': '손용호', '주연1': '마동석', '주연2': '김상중', '개봉일': '2019.09.11', '관객수': 4573902, '장르': '범죄/액션', '수상': '확인불가'}
누적완료:{'연도': 2019, '순위': 14, '제목': '82년생 김지영', '감독': '김도영', '주연1': '정유미', '주연2': '공유', '개봉일': '2019.10.23', '관객수': 3679265, '장르': '드라마', '수상': '황금촬영상'}
누적완료:{'연도': 2019, '순위': 15, '제목': '분노의 질주: 홉스&쇼', '감독': '데이비드 레이치', '주연1': '드웨인 존슨', '주연2': '제이슨 스타뎀', '개봉일': '2019.08.14', '관객수': 3657627, '장르': '액션', '수상': '확인불가'}
누적완료:{'연도': 2019, '순위': 16, '제목': '토이 스토리 4', '감독': '조시 쿨리', '주연1': '톰 행크스', '주연2': '팀 알렌', '개봉일': '2019.06.20', '관객수': 3400623, '장르': '애니메이션/어드벤처/코미디', '수상': '아카데미시상식'}
누적완료:{'연도': 2019, '순위': 17, '제목': '돈', '감독': '박누리', '주연1': '류준열', '주연2': '유지태', '개봉일': '2019.03.20', '관객수': 3389125, '장르': '범죄', '수상': '황금촬영상'}
누적완료:{'연도': 2019, '순위': 18, '제목': '악인전', '감독':

누적완료:{'연도': 2020, '순위': 14, '제목': '클로젯', '감독': '김광빈', '주연1': '하정우', '주연2': '김남길', '개봉일': '2020.02.05', '관객수': 1270907, '장르': '미스터리/드라마', '수상': '브뤼셀국제판타스틱영화제'}
누적완료:{'연도': 2020, '순위': 15, '제목': '오케이 마담', '감독': '이철하', '주연1': '엄정화', '주연2': '박성웅', '개봉일': '2020.08.12', '관객수': 1229372, '장르': '코미디/액션', '수상': '확인불가'}
누적완료:{'연도': 2020, '순위': 16, '제목': '해치지않아', '감독': '손재곤', '주연1': '안재홍', '주연2': '강소라', '개봉일': '2020.01.15', '관객수': 1224726, '장르': '코미디', '수상': '확인불가'}
누적완료:{'연도': 2020, '순위': 17, '제목': '천문: 하늘에 묻는다', '감독': '허진호', '주연1': '최민식', '주연2': '한석규', '개봉일': '2019.12.26', '관객수': 2000735, '장르': '시대극', '수상': '확인불가'}
누적완료:{'연도': 2020, '순위': 18, '제목': '결백', '감독': '박상현', '주연1': '신혜선', '주연2': '배종옥', '개봉일': '2020.06.10', '관객수': 894025, '장르': '드라마', '수상': '춘사국제영화제'}
누적완료:{'연도': 2020, '순위': 19, '제목': '1917', '감독': '샘 멘데스', '주연1': '조지 맥케이', '주연2': '딘 찰스 채프먼', '개봉일': '2020.02.19', '관객수': 882432, '장르': '전쟁/드라마', '수상': '다비드 디 도나텔로 어워드'}
누적완료:{'연도': 2020, '순위': 20, '제목': '작은 아씨들', '감독': '그레타 거윅', '주연1': '시얼샤

누적완료:{'연도': 2021, '순위': 15, '제목': '007 노 타임 투 다이', '감독': '캐리 조지 후쿠나가', '주연1': '다니엘 크레이그', '주연2': '라미 말렉', '개봉일': '2021.09.29', '관객수': 1229971, '장르': '액션', '수상': '아카데미시상식'}
누적완료:{'연도': 2021, '순위': 16, '제목': '미나리', '감독': '리 아이작 정', '주연1': '스티븐 연', '주연2': '한예리', '개봉일': '2021.03.03', '관객수': 1134788, '장르': '드라마', '수상': '춘사국제영화제'}
누적완료:{'연도': 2021, '순위': 17, '제목': '발신제한', '감독': '김창주', '주연1': '조우진', '주연2': '이재인', '개봉일': '2021.06.23', '관객수': 955883, '장르': '스릴러/드라마', '수상': '확인불가'}
누적완료:{'연도': 2021, '순위': 18, '제목': '보스 베이비 2', '감독': '톰 맥그래스', '주연1': '알렉 볼드윈', '주연2': '아미 세다리스', '개봉일': '2021.07.21', '관객수': 925277, '장르': '애니메이션/가족/어드벤처', '수상': '확인불가'}
누적완료:{'연도': 2021, '순위': 19, '제목': '콰이어트 플레이스 2', '감독': '존 크래신스키', '주연1': '에밀리 블런트', '주연2': '존 크래신스키', '개봉일': '2021.06.16', '관객수': 863339, '장르': '스릴러', '수상': '확인불가'}
누적완료:{'연도': 2021, '순위': 20, '제목': '랑종', '감독': '반종 피산다나쿤', '주연1': '나릴야 군몽콘켓', '주연2': '싸와니 우툼마', '개봉일': '2021.07.14', '관객수': 834534, '장르': '공포/스릴러/드라마', '수상': '부천국제판타스틱영화제'}
누적완료:{'연도': 2021

누적완료:{'연도': 2022, '순위': 16, '제목': '육사오(6/45)', '감독': '박규태', '주연1': '고경표', '주연2': '이이경', '개봉일': '2022.08.24', '관객수': 1981014, '장르': '코미디', '수상': '백상예술대상'}
누적완료:{'연도': 2022, '순위': 17, '제목': '헤어질 결심', '감독': '박찬욱', '주연1': '박해일', '주연2': '탕웨이', '개봉일': '2022.06.29', '관객수': 1904127, '장르': '로맨스/멜로/드라마', '수상': '백상예술대상'}
누적완료:{'연도': 2022, '순위': 18, '제목': '외계+인 1부', '감독': '최동훈', '주연1': '류준열', '주연2': '김우빈', '개봉일': '2022.07.20', '관객수': 1540630, '장르': '액션/판타지/SF', '수상': '황금촬영상'}
누적완료:{'연도': 2022, '순위': 19, '제목': '영웅', '감독': '윤제균', '주연1': '정성화', '주연2': '김고은', '개봉일': '2022.12.21', '관객수': 3274742, '장르': '드라마/뮤지컬', '수상': '대종상영화제'}
누적완료:{'연도': 2022, '순위': 20, '제목': '해적: 도깨비 깃발', '감독': '김정훈', '주연1': '강하늘', '주연2': '한효주', '개봉일': '2022.01.26', '관객수': 1339242, '장르': '어드벤처/액션/코미디', '수상': '확인불가'}
누적완료:{'연도': 2022, '순위': 21, '제목': '브로커', '감독': '고레에다 히로카즈', '주연1': '송강호', '주연2': '강동원', '개봉일': '2022.06.08', '관객수': 1261173, '장르': '드라마', '수상': '아시아필름어워즈'}
누적완료:{'연도': 2022, '순위': 22, '제목': '신비한 동물들과 덤블도어의 비밀', '감독': '데